In [1]:
## library imports

## db connection params
from config import get_engine

## IO
import pandas as pd

## linear algebra
import numpy as np

## regex 
import re 

## timimg
import time

from processing import strip, replace_empty_string,map_season,replace_finished_game_vals

In [2]:
## db engine
engine = get_engine("landing")
processing_engine = get_engine('processing')

In [3]:
## getting curling linescores from raspberry pi MySql DB

rankings = pd.read_sql("SELECT * FROM Ranking;",con=engine)

In [4]:
cols = ['Name','Rank']

In [5]:
## strip cols

rankings[cols] = rankings[cols].applymap(strip)

In [6]:
## replace empty strings with null values

rankings[cols] = rankings[cols].applymap(replace_empty_string)

In [7]:
## add rank

pat = r"(\d+)(?=.)"
ranks = rankings['Rank'].str.extract(pat=pat,expand=False)
assert ranks.isnull().sum() == 0
rankings['Rank'] = ranks.copy()

In [8]:
## get TeamID

pat = r"^.*teamid=(\d*)"
team_ids = rankings['TeamLink'].str.extract(pat=pat,expand=False)
assert team_ids.isnull().sum() == 0

rankings['TeamID'] = team_ids.copy()

In [9]:
## sql

index = False
if_exists = 'append'
rankings.to_sql("Ranking",con=processing_engine,index=index,if_exists=if_exists)